In [2]:
%matplotlib inline
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from IPython.display import clear_output
import time
from modules.factory import ohlc_chart

### 0. Parameters

In [3]:
seq_length = 180
data_dim = 4
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 1

### 1. Data Reconstruction

In [4]:
training_file = '../data/rnn_training_set-60day.csv'
xy = np.genfromtxt(training_file, delimiter=',', dtype=np.float32)


### 2. Training

In [5]:
# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

In [6]:
# build a network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

In [7]:
# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [8]:
# RMSE
targets = tf.placeholder(tf.float32, [None, 1])
predictions = tf.placeholder(tf.float32, [None, 1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))

In [9]:
sess = tf.Session()

In [10]:
sess.run(tf.global_variables_initializer())